In [1]:
!nvidia-smi

Thu Dec 25 14:26:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. Clone SAM và SAM 2 và cài thư viện

In [2]:
!git clone https://github.com/facebookresearch/segment-anything.git
!git clone https://github.com/facebookresearch/segment-anything-2.git

fatal: destination path 'segment-anything' already exists and is not an empty directory.
fatal: destination path 'segment-anything-2' already exists and is not an empty directory.


In [3]:
!pip install opencv-python matplotlib scikit-image tqdm

## 2. Tải checkpoint

In [4]:
# SAM
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

# SAM2
!wget https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt

--2025-12-25 14:26:28--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.33, 3.171.22.68, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth.2’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   370MB/s    in 6.7s    

2025-12-25 14:26:35 (367 MB/s) - ‘sam_vit_h_4b8939.pth.2’ saved [2564550879/2564550879]

--2025-12-25 14:26:35--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_base_plus.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.33, 3.171.22.68, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 323606802 (309M) [application/vnd.snesdev-page-

## 3. Import & khởi tạo model

In [6]:
import cv2, os, time
import numpy as np
import torch

## 3.1 SAM

In [7]:
from segment_anything import sam_model_registry, SamPredictor

sam = sam_model_registry["vit_h"](checkpoint="/kaggle/working/sam_vit_h_4b8939.pth").cuda()

sam_predictor = SamPredictor(sam)

## 3.2 SAM 2

In [8]:
%cd /kaggle/working/segment-anything-2 # direct vào folder sam2 đã clone
!pip install -e .

/kaggle/working/segment-anything-2
Obtaining file:///kaggle/working/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.4 MB/s eta 0:00:00
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp312-cp312-linux_x86_64.whl size=13868 sha256=024ba0819764e371a4d35918f4c90bf5624bfd9d17f950102c1214b0e17d0b69
  Stored in directory: /tmp/pip-ephem-wheel-cache-gqjji1jh/wheels/2b/2b/bf/6d22b3dc90bbf364a3f10ebca24459a83d58eafa29199af0ff
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=5d662eb8ead5423fcbf12fd18df7ddcc9da6777aab693462d4c4187571b5fbc4
  

In [9]:
import hydra
from hydra import initialize_config_dir
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

hydra.core.global_hydra.GlobalHydra.instance().clear()

initialize_config_dir(
    config_dir="/kaggle/working/segment-anything-2/sam2/configs", # direct vào folder configs trong sam2
    version_base=None
)

sam2 = build_sam2(
    config_file="sam2.1/sam2.1_hiera_l",
    ckpt_path="/kaggle/working/sam2.1_hiera_large.pt",
    device="cuda"
)

sam2_predictor = SAM2ImagePredictor(sam2)

## 4. Hàm inference

## 4.1 SAM

In [36]:
def run_sam(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    torch.cuda.synchronize()
    start = time.time()

    sam_predictor.set_image(img)
    h, w, _ = img.shape
    point = np.array([[w//2, h//2]])
    label = np.array([1])

    masks, _, _ = sam_predictor.predict(
        point_coords=point,
        point_labels=label,
        multimask_output=False
    )

    torch.cuda.synchronize()
    elapsed = time.time() - start

    mask = masks[0]
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    if mask.ndim == 3:
        mask = mask[0]

    return mask.astype(bool), elapsed


## 4.2 SAM 2

In [37]:
def run_sam2(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    torch.cuda.synchronize()
    start = time.time()
    
    sam2_predictor.set_image(img)

    h, w, _ = img.shape
    point = np.array([[w//2, h//2]])


    masks, _, _ = sam2_predictor.predict(
        point_coords=point,
        point_labels=[1],
        multimask_output=False
    )

    mask = masks[0]

    torch.cuda.synchronize()
    elapsed = time.time() - start
    
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    if mask.ndim == 3:
        mask = mask[0]

    return mask.astype(bool), elapsed


## 5. Metrics

In [38]:
def iou(pred, gt):
    inter = np.logical_and(pred, gt).sum()
    union = np.logical_or(pred, gt).sum()
    return inter / union if union > 0 else 0

def dice(pred, gt):
    inter = np.logical_and(pred, gt).sum()
    return 2 * inter / (pred.sum() + gt.sum() + 1e-6)

## 6. Evaluation

In [39]:
import os
import time
import pandas as pd
from tqdm import tqdm

def find_mask_path(mask_dir, image_name):

    base = os.path.splitext(image_name)[0]
    for ext in [".png", ".jpg", ".jpeg"]:
        p = os.path.join(mask_dir, base + ext)
        if os.path.exists(p):
            return p
    return None


def evaluate_dataset(dataset_name, eval_root):

    img_dir = os.path.join(eval_root, "img")
    mask_dir = os.path.join(eval_root, "mask")

    assert os.path.isdir(img_dir), f"Image folder not found: {img_dir}"
    assert os.path.isdir(mask_dir), f"Mask folder not found: {mask_dir}"

    img_names = sorted(os.listdir(img_dir))
    results = []

    skipped = 0

    for name in tqdm(img_names, desc=f"Evaluating {dataset_name}"):

        img_path = os.path.join(img_dir, name)
        mask_path = find_mask_path(mask_dir, name)
        if mask_path is None:
            print(f"[WARN] Missing mask for {name}, skipping.")
            skipped += 1
            continue

        gt = cv2.imread(mask_path, 0)

        if gt is None:
            print(f"[WARN] Missing mask: {mask_path}, skipping.")
            skipped += 1
            continue

        gt = gt > 0  # binarize
        gt = gt.astype(bool)

        sam_mask, t_sam = run_sam(img_path)

        sam2_mask, t_sam2 = run_sam2(img_path)

        iou_sam = iou(sam_mask, gt)
        iou_sam2 = iou(sam2_mask, gt)
        dice_sam = dice(sam_mask, gt)
        dice_sam2 = dice(sam2_mask, gt)

        results.append([
            name,
            iou_sam, iou_sam2,
            dice_sam, dice_sam2,
            t_sam, t_sam2
        ])

    df = pd.DataFrame(
        results,
        columns=[
            "Image",
            "IoU_SAM", "IoU_SAM2",
            "Dice_SAM", "Dice_SAM2",
            "Time_SAM", "Time_SAM2"
        ]
    )

    summary = df.mean(numeric_only=True)

    print(f"\n{dataset_name} summary:")
    display(summary)

    if skipped > 0:
        print(f"[INFO] Skipped {skipped} images due to missing masks.")

    return df, summary


In [19]:
DATASETS = {
    "COD10K": "/kaggle/input/sam-eval/COD10K_SPLIT/COD10K_SPLIT/eval",
    "DUTS": "/kaggle/input/sam-eval/DUTS_SPLIT/DUTS_SPLIT/eval",
    "ECSSD": "/kaggle/input/sam-eval/ECSSD_SPLIT/ECSSD_SPLIT/eval",
    "Kvasir": "/kaggle/input/sam-eval/Kvasir_VAL/Kvasir_VAL",
    "MSRA-B": "/kaggle/input/sam-eval/MSRA-B_SPLIT/MSRA-B_SPLIT/eval"
}

all_summaries = []

for name, path in DATASETS.items():
    _, summary = evaluate_dataset(name, path)
    summary["Dataset"] = name
    all_summaries.append(summary)

final_df = pd.DataFrame(all_summaries).set_index("Dataset")
display(final_df)

Evaluating COD10K: 100%|██████████| 456/456 [22:39<00:00,  2.98s/it]


COD10K summary:


IoU_SAM      0.288616
IoU_SAM2     0.385310
Dice_SAM     0.334912
Dice_SAM2    0.424578
Time_SAM     0.096972
Time_SAM2    0.696888
dtype: float64

Evaluating DUTS:   7%|▋         | 173/2335 [08:33<1:46:58,  2.97s/it]


KeyboardInterrupt: 